In [20]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

# Create Function statement

语法：
```
create [or replace] function function_name(param_list)
   returns return_type 
   language plpgsql
  as
$$
declare 
   -- variable declaration
begin
   -- logic
end;
$$;
```

In [21]:
sql = """
create function get_film_count(len_from int, len_to int)
returns int
language plpgsql
as
$$
declare
   film_count integer;
begin
   select count(*) 
   into film_count
   from film
   where length between len_from and len_to;
   
   return film_count;
end;
$$;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x284f2d1df30>

In [22]:
sql = """
select get_film_count(60, 90);
"""
run_sql(cursor, sql)

   get_film_count
0             229


In [23]:
sql = """
select get_film_count(
    len_from => 40, 
     len_to => 90
);
"""
run_sql(cursor, sql)

   get_film_count
0             325


## Function Parameter Modes

- IN: 默认模式，传入参数的值不会被函数改变
- OUT: 传出参数，函数可以改变传入参数的值
- INOUT: 传入传出参数，函数可以改变传入参数的值

In [24]:
# IN 模式
sql = """
create or replace function find_film_by_id(p_film_id int)
returns varchar
language plpgsql
as $$
declare
   film_title film.title%type;
begin
  -- find film title by id
  select title 
  into film_title
  from film
  where film_id = p_film_id;
  
  if not found then
     raise 'Film with id % not found', p_film_id;
  end if;
  
  return film_title;
end;$$
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x284f2d1df30>

In [25]:
sql = """
select * from find_film_by_id(1);
"""
run_sql(cursor, sql)

    find_film_by_id
0  Academy Dinosaur


In [26]:
# OUT 模式
sql = """
create or replace function get_film_stat(
    out min_len int,
    out max_len int,
    out avg_len numeric) 
language plpgsql
as $$
begin
  
  select min(length),
         max(length),
		 avg(length)::numeric(5,1)
  into min_len, max_len, avg_len
  from film;

end;$$
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x284f2d1df30>

In [27]:
sql = """
select * from get_film_stat();
"""
run_sql(cursor, sql)

   min_len  max_len avg_len
0       46      185   115.3


In [28]:
# INOUT 模式
sql = """
create or replace function swap(
	inout x int,
	inout y int
) 
language plpgsql	
as $$
begin
   select x,y into y,x;
end; $$;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x284f2d1df30>

In [29]:
sql = """
select * from swap(10,20);
"""
run_sql(cursor, sql)

    x   y
0  20  10


## Function Overloading

函数重载是指在同一个schema中，可以定义多个同名函数，只要函数的参数列表不同即可。

In [30]:
sql = """
create or replace function get_rental_duration(
	p_customer_id integer
)
returns integer 
language plpgsql
as $$
declare 
	rental_duration integer; 
begin
	select 
		sum( extract(day from return_date - rental_date)) 
	into rental_duration 
    from rental 
	where customer_id = p_customer_id;

	return rental_duration;
end; $$
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x284f2d1df30>

In [31]:
sql = """
SELECT get_rental_duration(232);
"""
run_sql(cursor, sql)

   get_rental_duration
0                   90


In [32]:
sql = """
create or replace function get_rental_duration(
	p_customer_id integer, 
	p_from_date date
)
returns integer 
language plpgsql
as $$
declare 
	rental_duration integer;
begin
	-- get the rental duration based on customer_id 
	-- and rental date
	select sum( extract( day from return_date + '12:00:00' - rental_date)) 
	into rental_duration
	from rental 
	where customer_id = p_customer_id and 
		  rental_date >= p_from_date;
	
	-- return the rental duration in days
	return rental_duration;
end; $$
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x284f2d1df30>

In [33]:
sql = """
SELECT get_rental_duration(232,'2005-07-01');
"""
run_sql(cursor, sql)

   get_rental_duration
0                   85


In [34]:
sql = """
create or replace function get_rental_duration(
	p_customer_id integer, 
	p_from_date date default '2005-01-01'
)
returns integer
language plpgsql
as $$
declare 
	rental_duration integer;
begin
	select sum( 
		extract( day from return_date + '12:00:00' - rental_date)
	) 
	into rental_duration
	from rental 
	where customer_id= p_customer_id and 
		  rental_date >= p_from_date;
	 
	return rental_duration;

end; $$
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x284f2d1df30>

In [35]:
sql = """
SELECT get_rental_duration(232);
"""
run_sql(cursor, sql)

AmbiguousFunction: 函数 get_rental_duration(integer) 不是唯一的
LINE 2: SELECT get_rental_duration(232);
               ^
HINT:  无法选择最佳候选函数. 你也许需要增加明确的类型转换.